In [1]:
import requests
import datetime as dt
import pandas as pd
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup

resp = requests.get('https://www.vox.com/a/sexual-harassment-assault-allegations-list')
soup = BeautifulSoup(resp.text, 'html.parser')

In [2]:
#predator們

predator_name_list = []

divs = soup.find_all('div','col')

for div in divs:
    names = div.find_all('a')
    for name in names:
        name = name.text.strip()
        #print(name)
        predator_name_list.append(name)
        #print(result1)
#print(result1)    

In [3]:
#何時被公諸於世

public_date_list = []

divs1 = soup.find_all('div','info-body')

for div in divs1:
    date_with_space = div.p.text.strip()
    #print(date_with_space)
    #print('\n')
    date_lists = date_with_space.split()
    #print(date_lists)
    #print('\n')
    date = ""
    for date_list in date_lists:
        date = date + date_list + " "
        date = date.replace('Publicly reported ',"")
    public_date_list.append(date)
    #print(date)
#print(result2)

In [4]:
#犯了什麼錯

crime_content_list = []

divs1 = soup.find_all('div','info-body')

for div in divs1:
    crimes = div.find_all('p')[1].text.strip()
    crime_content_list.append(crimes)
    #print(crimes)
#print(result3)

In [5]:
#受害者

victim_name_list = []

victims = soup.find_all('footer')
#print(op)
for victim in victims:
    
    try:
        name = victim.strong.text.strip().replace(',','')
        #print(victim.a)
        #print(victim.a.text.strip())
        victim_name_list.append(name)
    
    except:
        if victim.a.text.strip() == "":
            continue
        else:
            name = victim.a.text.strip().replace(',','')
            victim_name_list.append(name)
            


In [6]:
#受害者職業

victim_career_list = []

victims = soup.find_all('footer')
#print(op)

for victim in victims:
    
        #name = victim.strong.text.strip().replace(',','')
        #print(victim.a)
        careers = victim.a.text.strip().split(", ")

        try:
            victim_career_list.append(careers[1])
            #print(careers[1])
        except:
            
            if careers[0] == "":
                continue
            else:
                victim_career_list.append('Unknown')
                #print('unknown')

In [7]:
#加害者職業

predators = soup.find_all('h5')
predator_career_list = []

for predator in predators:
     predator_career = predator.text.strip()
     predator_career_list.append(predator_career)    


In [8]:
#連結

sources = soup.find_all('p','small')

person_all_links = ""
n = 0
more_info = []

for source in sources:
        source = source.find_all('a','ref')
        link_of_s = []
        for s in source:
                
                alllink = s['href']
                strlink = str(alllink)
                link_of_s.append(strlink)
        
        for link in link_of_s:
                
                n=n+1
                if len(link_of_s) == 1:
                        person_all_links = link+'\n'
                        
                elif len(link_of_s)>1:
                        if n>1:
                                person_all_links = person_all_links+link+'\n'
                                
                        elif n == 1:
                                person_all_links = link+'\n'
                                
        n=0                
        
        more_info.append(person_all_links)
        
        
print(len(more_info))
        

262


In [9]:
#組成dataframe

data = {'predator':predator_name_list,
        'predator career':predator_career_list,
        'victim':victim_name_list,
        'victum career':victim_career_list, 
        'Public date':public_date_list,
        'crime content':crime_content_list,
        'source/more info':more_info}

df = pd.DataFrame(data) 

df.head()

,predator,predator career,victim,victum career,Public date,crime content,source/more info
0,Frankie Shaw,"Creator and star, Showtime's SMILF",anonymous,SMILF staffer,"December 17, 2018",Multiple staffers have said she mishandled sex...,https://www.hollywoodreporter.com/news/smilf-c...
1,Michael Weatherly,"Actor, CBS's Bull",Eliza Dushku,actress,"December 13, 2018",A co-star says he made inappropriate comments ...,https://www.nytimes.com/2018/12/13/business/me...
2,Steven Wilder Striegel,Actor,Paige Carnes,who reported that Striegel abused her,"September 6, 2018",A woman has said he sexually abused her when s...,http://www.latimes.com/entertainment/movies/la...
3,Gerard Depardieu,Actor,the New York Magazine vertical The Cut,Unknown,"August 30, 2018",An actress has said he raped her. French autho...,https://www.bbc.com/news/world-europe-45360902...
4,Chase Finlay,"Former principal dancer, New York City Ballet",lawsuit filed by Alexandra Waterbury against N...,Unknown,"August 28, 2018",A woman said he shared naked pictures of her w...,https://www.nytimes.com/2018/08/28/arts/dance/...


In [55]:
import sqlite3
  
conn = sqlite3.connect('MeToo.db')  #建立資料庫
cursor = conn.cursor()
cursor.execute('DROP TABLE rawdata') #直接砍掉舊Table，因為有時候會有新增column什麼的
cursor.execute('CREATE TABLE rawdata(predator, "predator career", victim, "victum career", "Public date", "crime content", "source/more info")')  #建立資料表
conn.commit()


In [56]:
#如果資料表存在，就寫入資料，否則建立資料表
df.to_sql('rawdata', conn, if_exists='append', index=False) 
 
#透過SQL語法讀取資料庫中的資料
us_df = pd.read_sql("SELECT * FROM rawdata WHERE victim='anonymous'", conn)
print(us_df)

                predator                                    predator career  \
0           Frankie Shaw                 Creator and star, Showtime's SMILF   
1         Nicholas Nixon  Photographer; former photography professor, Ma...   
2   Patrick Demarchelier                                       Photographer   
3           Karl Templer                                            Stylist   
4    Vincent Cirrincione                                     Talent manager   
5               Stan Lee  Comic book writer; former editor-in-chief, Mar...   
6            Paul Haggis                             Director, screenwriter   
7            T.J. Miller                                    Actor, comedian   
8          John Lasseter       CEO, Pixar and Walt Disney Animation Studios   
9              Andy Dick                                 Actor and comedian   
10          Kevin Spacey                                              Actor   
11             Ken Baker                       Senio